In [17]:
import re

In [2]:
import torch
import torch.nn as nn

In [3]:
from transformers import DistilBertTokenizer, DistilBertModel

In [4]:
import torchsummary

In [8]:

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [9]:
def insert_module(model, indices, modules):
    indices = indices if isinstance(indices, list) else [indices]
    modules = modules if isinstance(modules, list) else [modules]
    assert len(indices) == len(modules)

    layers_name = [name for name, _ in model.named_modules()][1:]
    for index, module in zip(indices, modules):
        layer_name = re.sub(r'(.)(\d)', r'[\2]', layers_name[index])
        exec("model.{name} = nn.Sequential(model.{name}, module)".format(name = layer_name))

In [33]:
class encoder(nn.Module):

    def __init__(self):
    
        super().__init__()

        self.pretrained = DistilBertModel.from_pretrained("distilbert-base-uncased")


    def forward(self, input):

        return self.pretrained(input)


In [14]:
layers_name = [name for name, _ in model.pretrained.named_modules()][1:]

In [15]:
layers_name

['embeddings',
 'embeddings.word_embeddings',
 'embeddings.position_embeddings',
 'embeddings.LayerNorm',
 'embeddings.dropout',
 'transformer',
 'transformer.layer',
 'transformer.layer.0',
 'transformer.layer.0.attention',
 'transformer.layer.0.attention.dropout',
 'transformer.layer.0.attention.q_lin',
 'transformer.layer.0.attention.k_lin',
 'transformer.layer.0.attention.v_lin',
 'transformer.layer.0.attention.out_lin',
 'transformer.layer.0.sa_layer_norm',
 'transformer.layer.0.ffn',
 'transformer.layer.0.ffn.dropout',
 'transformer.layer.0.ffn.lin1',
 'transformer.layer.0.ffn.lin2',
 'transformer.layer.0.ffn.activation',
 'transformer.layer.0.output_layer_norm',
 'transformer.layer.1',
 'transformer.layer.1.attention',
 'transformer.layer.1.attention.dropout',
 'transformer.layer.1.attention.q_lin',
 'transformer.layer.1.attention.k_lin',
 'transformer.layer.1.attention.v_lin',
 'transformer.layer.1.attention.out_lin',
 'transformer.layer.1.sa_layer_norm',
 'transformer.layer.1.

In [34]:
class sparse_part(nn.Module):

    def __init__(self, in_dim, out_dim):

        super().__init__()

        self.lin = nn.Linear(in_features=in_dim, out_features=out_dim)
        self.laynorm = nn.LayerNorm(out_dim)
        self.act = nn.ReLU6()

    def forward(self, input):

        x = self.lin(input)
        x = self.laynorm(self.act(x))

        return x

In [6]:
class sparse_module(nn.Module):
    
    def __init__(self, layer_size: int, internal_size: int):

        super().__init__()

        self.layer = sparse_part(layer_size, internal_size)

        self.bootleneck = sparse_part(internal_size, internal_size//8)

        self.output = nn.Linear(internal_size//8, layer_size)

    def forward(self, input):

        x = self.layer(input)
        x = self.bottleneck(self.lin2(input))
        x = nn.functional.leaky_relu(self.output(x))

        return x, nn.functional.l1_loss(x)

In [35]:
model = encoder()

In [36]:
insert_module(model.pretrained, 33, sparse_module(768, 96))

In [37]:
list(model.pretrained.named_modules())

[('',
  DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_features=768, out_feat

In [29]:
encoded_input["input_ids"].shape

torch.Size([1, 12])

In [30]:
encoded_input

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [105]:
list(encoded_input.values())

[tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])]

In [77]:
[val.shape for val in  encoded_input.values()]

[torch.Size([1, 12]), torch.Size([1, 12])]

In [38]:
torchsummary.summary(model, input_size=[(512, )],
                    device="cpu")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


IndexError: tuple index out of range

In [81]:
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li